In [8]:
import torch
import torchvision.transforms as transforms
import torchvision.models.video as models
import cv2
import numpy as np
from PIL import Image
from imblearn.over_sampling import SMOTE
import re
import os
import json
import pickle
from sklearn.preprocessing import LabelEncoder

Leemos los archivos de Features y los metemos en un diccionario ordenado 

In [9]:
with open('F:/MVFoulRecognition/features/Mvit/train/end/train_combinados.pkl', 'rb') as file:  # 'rb' es lectura en modo binario
    loaded_data = pickle.load(file)

resultado_train = {k: v for dic in loaded_data for k, v in dic.items()}

dic_ordenado_train = dict(sorted(resultado_train.items(), key=lambda x: int(x[0].split('_')[1])))

#print(dic_ordenado_train)
with open('F:/MVFoulRecognition/features/Mvit/test/test_combined.pkl', 'rb') as file:  # 'rb' es lectura en modo binario
    loaded_data = pickle.load(file)

resultado_test = {k: v for dic in loaded_data for k, v in dic.items()}

# Salida: {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6}
dic_ordenado_test = dict(sorted(resultado_test.items(), key=lambda x: int(x[0].split('_')[1])))


Funcion para cargar las etiquetas desde un archivo JSON

In [10]:
def load_action_labels(json_path):

    with open(json_path, 'r') as f:
        data = json.load(f)
    
    action_labels = {}
    for action_id, action_data in data['Actions'].items():
        label = action_data['Offence']  # Extraer la etiqueta de 'Action class'
        action_name = f"action_{action_id}"  # Formar el nombre de la acción
        action_labels[action_name] = label
    
    return action_labels

In [11]:
ls = load_action_labels("F:/data/mvfouls/train/annotations.json")
ls = sorted(ls.items())
sorted_labels_train = sorted(ls, key=lambda x: int(x[0].split('_')[1]))

ls = load_action_labels("F:/data/mvfouls/test/annotations.json")
ls = sorted(ls.items())
sorted_labels_test = sorted(ls, key=lambda x: int(x[0].split('_')[1]))



In [12]:
#Añadimos las etiquetas de los videos extendidos
sorted_labels_train.extend([('action_' + str(i), 'No offence') for i in range(2916, 3945)])

Comprobamos y eliminamos las acciones que se han procesado mal

In [13]:

diccionario = {i: 0 for i in range(0, 3945)}

# Recorrer las claves en `res`
for r in dic_ordenado_train:
    # Extraer el número de la clave 'action_X'
    action = int(r.split("_")[1])  # Convertimos el número extraído a entero
    
    # Si el número extraído de `action_X` está en `diccionario`, lo marcamos como 1
    if action in diccionario:
        diccionario[action] = 1

# Mostrar el diccionario actualizado
diccionario_filtrado = {k: v for k, v in diccionario.items() if v == 0}
print(diccionario_filtrado)

{3266: 0, 3569: 0}


In [14]:
sorted_labels_train = [item for item in sorted_labels_train if item[0] != 'action_3266' and item[0] != 'action_3569' ]

In [15]:
for item in sorted_labels_train:
    if item[0] == 'action_3569':
        print(item)

Cargamos y estructuramos los datos de entrada del modelo de clasificación 

In [16]:
# 1. Filtrar las acciones con etiqueta 'Offence'
offence_actions = [item for item in sorted_labels_train if item[1] == 'Offence']

# 2. Seleccionar las primeras mil acciones con etiqueta 'Offence'
# Aseguramos que hay al menos mil acciones con etiqueta 'Offence'
actions_to_remove = offence_actions[:1100]

# 3. Guardar estas mil acciones en una lista
actions_to_remove_list = actions_to_remove

sorted_labels_train = [item for item in sorted_labels_train if item not in actions_to_remove_list]

In [17]:
actions_to_remove_list

[('action_0', 'Offence'),
 ('action_1', 'Offence'),
 ('action_2', 'Offence'),
 ('action_3', 'Offence'),
 ('action_4', 'Offence'),
 ('action_5', 'Offence'),
 ('action_6', 'Offence'),
 ('action_7', 'Offence'),
 ('action_8', 'Offence'),
 ('action_9', 'Offence'),
 ('action_10', 'Offence'),
 ('action_11', 'Offence'),
 ('action_12', 'Offence'),
 ('action_13', 'Offence'),
 ('action_14', 'Offence'),
 ('action_15', 'Offence'),
 ('action_16', 'Offence'),
 ('action_17', 'Offence'),
 ('action_18', 'Offence'),
 ('action_20', 'Offence'),
 ('action_21', 'Offence'),
 ('action_22', 'Offence'),
 ('action_23', 'Offence'),
 ('action_24', 'Offence'),
 ('action_26', 'Offence'),
 ('action_27', 'Offence'),
 ('action_28', 'Offence'),
 ('action_29', 'Offence'),
 ('action_30', 'Offence'),
 ('action_31', 'Offence'),
 ('action_34', 'Offence'),
 ('action_35', 'Offence'),
 ('action_36', 'Offence'),
 ('action_37', 'Offence'),
 ('action_39', 'Offence'),
 ('action_41', 'Offence'),
 ('action_42', 'Offence'),
 ('action_4

In [18]:
for action in actions_to_remove_list:
    dic_ordenado_train.pop(action[0], None) 
# Ver el diccionario después de la eliminación
print("Diccionario después de eliminar las acciones:", dic_ordenado_train.keys())

Diccionario después de eliminar las acciones: dict_keys(['action_19', 'action_25', 'action_32', 'action_33', 'action_38', 'action_40', 'action_55', 'action_58', 'action_63', 'action_85', 'action_98', 'action_121', 'action_132', 'action_151', 'action_162', 'action_176', 'action_193', 'action_203', 'action_214', 'action_220', 'action_223', 'action_224', 'action_228', 'action_231', 'action_237', 'action_238', 'action_241', 'action_242', 'action_243', 'action_246', 'action_252', 'action_253', 'action_254', 'action_259', 'action_269', 'action_277', 'action_301', 'action_325', 'action_328', 'action_331', 'action_345', 'action_349', 'action_352', 'action_359', 'action_369', 'action_371', 'action_373', 'action_380', 'action_390', 'action_394', 'action_407', 'action_424', 'action_443', 'action_445', 'action_449', 'action_456', 'action_461', 'action_473', 'action_476', 'action_477', 'action_484', 'action_486', 'action_491', 'action_497', 'action_501', 'action_505', 'action_506', 'action_507', 'a

In [19]:
sorted_labels_train

[('action_19', 'No offence'),
 ('action_25', 'No offence'),
 ('action_32', 'No offence'),
 ('action_33', 'No offence'),
 ('action_38', 'No offence'),
 ('action_40', 'No offence'),
 ('action_55', 'No offence'),
 ('action_58', 'No offence'),
 ('action_63', 'No offence'),
 ('action_85', 'No offence'),
 ('action_98', 'No offence'),
 ('action_121', 'No offence'),
 ('action_132', 'No offence'),
 ('action_151', 'No offence'),
 ('action_162', 'No offence'),
 ('action_176', 'No offence'),
 ('action_193', 'No offence'),
 ('action_203', 'No offence'),
 ('action_214', 'No offence'),
 ('action_220', 'No offence'),
 ('action_223', 'No offence'),
 ('action_224', 'No offence'),
 ('action_228', 'No offence'),
 ('action_231', 'No offence'),
 ('action_237', 'No offence'),
 ('action_238', 'No offence'),
 ('action_241', 'No offence'),
 ('action_242', 'No offence'),
 ('action_243', 'No offence'),
 ('action_246', 'No offence'),
 ('action_252', 'No offence'),
 ('action_253', 'No offence'),
 ('action_254', 'No

In [20]:
len(dic_ordenado_train)

2843

In [21]:
X_train  =np.array(list(dic_ordenado_train.values()))
labels_dic_train = dict(sorted_labels_train)
Y_train = np.array(list(labels_dic_train.values()))

print(np.unique(Y_train)) 
X_test = np.array(list(dic_ordenado_test.values()))
labels_dict_test = dict(sorted_labels_test)
Y_test = np.array(list(labels_dict_test.values()))


#X_valid = np.array(list(dic_ordenado_valid.values()))
#labels_dict_valid = dict(sorted_labels_valid)
#Y_valid = np.array(list(labels_dict_valid.values()))

print("Ejemplo de etiquetas en Y_train:", Y_train[:10])
print("Ejemplo de etiquetas en Y_test:", Y_test[:10])
#print("Ejemplo de etiquetas en Y_valid:", Y_valid[:10])
label_encoder = LabelEncoder()

# Ajustamos el codificador con TODAS las etiquetas posibles
label_encoder.fit(np.concatenate([Y_train, Y_test]))

# Transformamos las etiquetas de texto a números
Y_train_encoded = label_encoder.transform(Y_train)
Y_test_encoded = label_encoder.transform(Y_test)
#Y_valid_encoded = label_encoder.transform(Y_valid)

print("Etiquetas originales:", np.unique(Y_train)) 
print("Etiquetas codificadas:", np.unique(Y_train_encoded))

print("Diccionario de conversión:", dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))

['No offence' 'Offence']
Ejemplo de etiquetas en Y_train: ['No offence' 'No offence' 'No offence' 'No offence' 'No offence'
 'No offence' 'No offence' 'No offence' 'No offence' 'No offence']
Ejemplo de etiquetas en Y_test: ['Offence' 'Offence' 'Offence' 'Offence' 'Offence' 'Offence' 'No offence'
 'Offence' 'Offence' 'No offence']
Etiquetas originales: ['No offence' 'Offence']
Etiquetas codificadas: [0 1]
Diccionario de conversión: {np.str_('No offence'): np.int64(0), np.str_('Offence'): np.int64(1)}


In [22]:
print(f"X_train type: {type(X_train)}, Y_train type: {type(Y_train)}")
print(f"X_test type: {type(X_test)}, Y_valid type: {type(Y_test)}")
#print(f"X_valid type: {type(X_valid)}, Y_valid type: {type(Y_valid)}")

# Verificar las dimensiones
print(f"X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}")
print(f"X_test shape: {X_test.shape}, Y_test shape: {Y_test.shape}")
#print(f"X_valid shape: {X_valid.shape}, Y_valid shape: {Y_valid.shape}")



X_train type: <class 'numpy.ndarray'>, Y_train type: <class 'numpy.ndarray'>
X_test type: <class 'numpy.ndarray'>, Y_valid type: <class 'numpy.ndarray'>
X_train shape: (2843, 1, 400), Y_train shape: (2843,)
X_test shape: (301, 1, 400), Y_test shape: (301,)


MODELO XGBOOST CON OPTIMIZACIÓN DE LOS HIPERPARÁMETROS


In [23]:
# Aplanar las dimensiones de los datos (como antes)
X_train_flattened = X_train.squeeze(1)  # (2916, 400)
X_test_flattened = X_test.squeeze(1)    # (301, 400)

# Aplicar SMOTE para balancear las clases en el conjunto de entrenamiento
#smote = SMOTE(random_state=42)
#X_train_resampled, Y_train_resampled = smote.fit_resample(X_train_flattened, Y_train_encoded)
print(np.unique(Y_train, return_counts=True))


(array(['No offence', 'Offence'], dtype='<U10'), array([1448, 1395]))


In [185]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from scipy.stats import uniform
import numpy as np

# Suponiendo que estos son tus datos completos
X_total = np.concatenate((X_train_flattened, X_test_flattened))
Y_total = np.concatenate((Y_train_encoded, Y_test_encoded))

# Nueva división estratificada
X_train, X_test, Y_train, Y_test = train_test_split(
    X_total, Y_total,
    test_size=0.2,         # o el porcentaje que quieras
    stratify=Y_total,      # 👈 esto asegura la misma proporción de clases
    random_state=42
)

# Definir el espacio de búsqueda de hiperparámetros
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],  # Número de árboles
    'learning_rate': uniform(0.01, 0.2),         # Tasa de aprendizaje
    'max_depth': [3, 4, 5, 6, 7],                # Profundidad máxima de los árboles
    'min_child_weight': [1, 2, 3, 4],            # Peso mínimo de los hijos
    'subsample': uniform(0.6, 0.4),              # Submuestra de las instancias
    'colsample_bytree': uniform(0.6, 0.4),       # Submuestra de las columnas
    'gamma': [0, 0.1, 0.2, 0.3],                 # Penalización por complejidad
    'class_weight': ['balanced', None],          # Manejo del desbalance
}

# Definir el modelo XGBoost
xgb = XGBClassifier(random_state=42)

# Realizar Randomized Search
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=30,  # Número de combinaciones a probar
    scoring='f1_macro',  # Usar F1 macro como métrica de optimización
    cv=3,  # Validación cruzada
    random_state=42,
    verbose=2,
    n_jobs=-1  # Para usar todos los núcleos del procesador
)

# Ajustar al conjunto de entrenamiento
random_search.fit(X_train, Y_train)

# Resultados de los mejores parámetros
print(f"Mejores parámetros encontrados: {random_search.best_params_}")

# Usar el mejor modelo
best_xgb = random_search.best_estimator_

# Realizar predicciones en el conjunto de prueba
Y_test_pred = best_xgb.predict(X_test)

# Evaluar el modelo
print(f"Precisión del modelo optimizado: {accuracy_score(Y_test, Y_test_pred)}")
print("Matriz de Confusión:")
print(confusion_matrix(Y_test, Y_test_pred))
print("Reporte de Clasificación:")
print(classification_report(Y_test, Y_test_pred))


Fitting 3 folds for each of 30 candidates, totalling 90 fits
Mejores parámetros encontrados: {'class_weight': None, 'colsample_bytree': np.float64(0.9754210836063001), 'gamma': 0.1, 'learning_rate': np.float64(0.04636499344142013), 'max_depth': 7, 'min_child_weight': 1, 'n_estimators': 400, 'subsample': np.float64(0.8446612641953124)}
Precisión del modelo optimizado: 0.8267090620031796
Matriz de Confusión:
[[217  80]
 [ 29 303]]
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.88      0.73      0.80       297
           1       0.79      0.91      0.85       332

    accuracy                           0.83       629
   macro avg       0.84      0.82      0.82       629
weighted avg       0.83      0.83      0.82       629



In [186]:
import pickle

with open('best_xgb_model_mvit.pkl', 'wb') as f:
    pickle.dump(best_xgb, f)